In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
import os

In [4]:
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
LANGCHAIN_TRACING_V2 = os.getenv("LANGCHAIN_TRACING_V2")
LANGCHAIN_ENDPOINT = os.getenv("LANGCHAIN_ENDPOINT")
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")
LANGCHAIN_PROJECT = os.getenv("LANGCHAIN_PROJECT")
SERPER_API_KEY = os.getenv("SERPER_API_KEY")
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
HUGGINGFACE_API_KEY = os.getenv("HUGGINGFACE_API_KEY")

In [5]:
os.environ["TAVILY_API_KEY"] = TAVILY_API_KEY
os.environ["LANGCHAIN_TRACING_V2"] = LANGCHAIN_TRACING_V2
os.environ["LANGCHAIN_ENDPOINT"] = LANGCHAIN_ENDPOINT
os.environ["LANGCHAIN_API_KEY"] = LANGCHAIN_API_KEY
os.environ["LANGCHAIN_PROJECT"] = LANGCHAIN_PROJECT
os.environ["SERPER_API_KEY"] = SERPER_API_KEY
os.environ["GEMINI_API_KEY"] = GEMINI_API_KEY
os.environ["GROQ_API_KEY"] = GROQ_API_KEY
os.environ["HUGGINGFACE_API_KEY"] = HUGGINGFACE_API_KEY

In [6]:
# with google gemini
"""from langchain_google_genai import GoogleGenerativeAIEmbeddings 
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
from langchain_google_genai import ChatGoogleGenerativeAI
llm  = ChatGoogleGenerativeAI(model ="gemini-1.0-pro")"""

'from langchain_google_genai import GoogleGenerativeAIEmbeddings \nembeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")\nfrom langchain_google_genai import ChatGoogleGenerativeAI\nllm  = ChatGoogleGenerativeAI(model ="gemini-1.0-pro")'

In [7]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
from langchain_groq import ChatGroq
llm=ChatGroq(model_name="Gemma2-9b-It")

c:\Users\mailm\Documents\langgraph-end-to-end\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Simple AI Assistant

In [8]:
while True:
    question=input("type you question. If you want to quit the chat write quit ")
    if question !="quit":
        print(llm.invoke(question).content)
    else:
        print("goodbye take care of yourself")
        break 


Hello! 👋  How can I help you today?

Imagine you're teaching a dog a new trick. 

You show the dog what to do, reward it when it gets close, and correct it when it makes a mistake. Over time, the dog learns the trick by recognizing patterns and understanding what gets it rewarded.

Machine learning is similar! 

Instead of a dog, we have a computer program. Instead of tricks, we have tasks like recognizing pictures, understanding language, or predicting the weather.

We "train" the program by giving it lots of data (like showing the dog examples of the trick). The program learns from this data, finding patterns and relationships, and gets better at the task over time. 

The more data it sees, the better it gets! 

It's like the computer is learning on its own, without being explicitly programmed for every single scenario.

SpaceX is a private American aerospace manufacturer and space transportation services company founded by Elon Musk in 2002. 

Here are some key things to know about 

In [9]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.chat_history  import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.messages import AIMessage


In [11]:
store = {}

In [12]:
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]


In [13]:
config = {"configurable": {"session_id": "firstchat"}}

In [14]:
model_with_memory= RunnableWithMessageHistory(llm, get_session_history)

In [15]:
model_with_memory.invoke(("Hi! I'm bob"), config=config).content

"Hi Bob! 👋\n\nIt's nice to meet you. What can I do for you today? 😊\n"

In [16]:
model_with_memory.invoke(("tell me what is my name?"), config = config).content

'Your name is Bob!  You told me at the beginning. 😊 \n'

In [17]:
store

{'firstchat': InMemoryChatMessageHistory(messages=[HumanMessage(content="Hi! I'm bob", additional_kwargs={}, response_metadata={}), AIMessage(content="Hi Bob! 👋\n\nIt's nice to meet you. What can I do for you today? 😊\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 15, 'total_tokens': 40, 'completion_time': 0.045454545, 'prompt_time': 8.977e-05, 'queue_time': 0.022567728000000002, 'total_time': 0.045544315}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-120e12b8-4538-4c4e-95a1-df42bf15268b-0', usage_metadata={'input_tokens': 15, 'output_tokens': 25, 'total_tokens': 40}), HumanMessage(content='tell me what is my name?', additional_kwargs={}, response_metadata={}), AIMessage(content='Your name is Bob!  You told me at the beginning. 😊 \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 55, 'total_tokens': 73, 

## RAG with LCEL

In [38]:
from langchain_community.document_loaders import TextLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain import PromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser


In [39]:
from langchain.document_loaders import PyPDFLoader

# Path to the PDF file
loader = PyPDFLoader("../data/attention.pdf")

# Asynchronous loading of pages
pages = []

async def load_pages():
    async for page in loader.alazy_load():
        pages.append(page)

# If running in an async environment, call load_pages
# For example, in an async function or Jupyter notebook with asyncio:
# await load_pages()


In [40]:
# creating chunks using RecursiveTextSplitter
docs = loader.load()
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=50,
    chunk_overlap=10,
    length_function=len
)

new_docs = text_splitter.split_documents(documents=docs)
doc_strings = [doc.page_content for doc in new_docs]


In [42]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
from langchain_groq import ChatGroq
llm=ChatGroq(model_name="Gemma2-9b-It")

In [43]:
db = Chroma.from_documents(new_docs, embeddings)
retriever = db.as_retriever(search_kwargs={"k":4})

In [55]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = PromptTemplate.from_template(template)

In [56]:
retrieval_chain = (
    RunnableParallel({"context": retriever, "question": RunnablePassthrough()})
    | prompt
    | llm
    | StrOutputParser()
    )

In [57]:
question = "What is attention mechanism? can you highlight 3 important points?"
print(retrieval_chain.invoke(question))

Based on the provided context, we know that "attention mechanisms" is a topic discussed in a PDF file named "attention.pdf".  

However, the text doesn't explain what an attention mechanism *is* or provide specific important points about it. All we know is that it's mentioned repeatedly on page 1 of the document and visually represented as "Figure 3" on page 12.  


To answer your question about the definition and important points, I'd need more information from the "attention.pdf" document. 




## Tools and Agents

In [58]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper


In [59]:
api_wrapper = WikipediaAPIWrapper()

In [60]:
tool = WikipediaQueryRun(api_wrapper=api_wrapper)

In [61]:
tool.name

'wikipedia'

In [62]:
tool.args

{'query': {'description': 'query to look up on wikipedia',
  'title': 'Query',
  'type': 'string'}}

In [63]:
print(tool.run({"query": "Elon Musk"}))

Page: Elon Musk
Summary: Elon Reeve Musk (; born June 28, 1971) is a businessman known for his key roles in the space company SpaceX and the automotive company Tesla, Inc. Other involvements include ownership of X Corp., the company that operates the social media platform X (formerly Twitter), and his role in the founding of the Boring Company, xAI, Neuralink, and OpenAI. In November 2024, president-elect Donald Trump announced Musk is set to become co-chair of the proposed Department of Government Efficiency (DOGE) in the second Trump administration. Musk is the wealthiest individual in the world; as of November 2024 Forbes estimates his net worth to be US$304 billion.
Musk was born in Pretoria, South Africa, and briefly attended the University of Pretoria before immigrating to Canada at the age of 18, acquiring citizenship through his Canadian-born mother. Two years later, he matriculated at Queen's University at Kingston in Canada. Musk later transferred to the University of Pennsyl

In [64]:
from langchain_community.tools import YouTubeSearchTool

In [65]:
tool2 = YouTubeSearchTool()

In [67]:
tool2.name

'youtube_search'

In [68]:
print(tool2.run("Johnny Haris"))

['https://www.youtube.com/watch?v=Cl_JpCSvTpk&pp=ygUMSm9obm55IEhhcmlz', 'https://www.youtube.com/watch?v=Fa1_VWuIElE&pp=ygUMSm9obm55IEhhcmlz']


In [69]:
from langchain_community.tools.tavily_search import TavilySearchResults

In [70]:
tool3 = TavilySearchResults()

In [74]:
print(tool3.run({"query": "Give some information about John F Kennedy's speech at rice university"}))

[{'url': 'https://en.wikipedia.org/wiki/We_choose_to_go_to_the_Moon', 'content': 'Address at Rice University on the Nation\'s Space Effort, commonly known by the sentence in the middle of the speech "We choose to go to the Moon", was a speech on September 12, 1962 by John F. Kennedy, the President of the United States.The aim was to bolster public support for his proposal to land a man on the Moon before 1970 and bring him safely back to Earth.'}, {'url': 'https://archives.library.rice.edu/repositories/2/resources/666', 'content': "President John F. Kennedy visited Rice University campus on September 12, 1962, and delivered a speech in the stadium on the Nation's Space Effort. Kennedy cites scientific progress as evidence that exploration of space is inevitable and argues that the United States should lead the space effort in order to retain a position of leadership on"}, {'url': 'https://www.rice.edu/jfk-speech', 'content': 'About Rice Give to Rice "We choose to go to the Moon", offic

In [75]:
from langchain.agents import AgentType
from langchain.agents import load_tools
from langchain.agents import initialize_agent

In [76]:
tool = load_tools(["wikipedia"], llm=llm)

In [78]:
agent = initialize_agent(tool, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

In [79]:
agent.run("What are some capabilities of SpaceX Starship")

C:\Users\mailm\AppData\Local\Temp\ipykernel_13048\3998087941.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  agent.run("What are some capabilities of SpaceX Starship")




> Entering new AgentExecutor chain...
Thought: I need to find information about SpaceX Starship on Wikipedia. 
Action: wikipedia
Action Input: SpaceX Starship
Observation: Page: SpaceX Starship
Summary: Starship is a two-stage fully reusable super heavy-lift launch vehicle under development by American aerospace company SpaceX. On April 20, 2023, with the first Integrated Flight Test, Starship became the most massive and most powerful vehicle ever to fly. SpaceX has developed Starship with the intention of lowering launch costs using economies of scale, aiming to achieve this by reusing both rocket stages by "catching" them with the launch tower's systems, increasing payload mass to orbit, increasing launch frequency, mass-manufacturing the rockets and adapting it to a wide range of space missions. Starship is the latest project in SpaceX's reusable launch system development program and plan to colonize Mars.
Starship's two stages are the Super Heavy booster and the Starship spacecra

'SpaceX Starship is designed to be a fully reusable super heavy-lift launch vehicle capable of transporting both crew and cargo to a variety of destinations, including Earth orbit, the Moon, and Mars. It is designed to be reusable and capable of landing propulsively. It will also be capable of point-to-point transport on Earth and refueling other Starship spacecraft.'